In [34]:
import pandas as pd
import numpy as np
import torch
from torch import nn, Tensor

In [35]:
def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
learning_rate=0.005
input_size=20
num_epochs=1000

In [42]:

data_csv=pd.read_csv("industry_ret_data_1500.csv")
dataset = data_csv.values
dataset=np.delete(dataset,0,axis=1)
dataset = dataset.astype('float32')
dataset=standardization(dataset)
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)
data_X, data_Y = create_dataset(dataset,input_size )
train_X = data_X[:1000]
train_Y = data_Y[:1000]
val_X=data_X[1000:1200]
val_Y=data_Y[1000:1200]
test_X = data_X[1200:]
test_Y = data_Y[1200:]
print(np.var(test_Y))
train_X = train_X.reshape(-1, 433, input_size )
train_Y = train_Y.reshape(-1,433,1)
test_X = test_X.reshape(-1,433, input_size )
test_Y=test_Y.reshape(-1,433,1)
val_X = val_X.reshape(-1, 433, input_size )
val_Y = val_Y.reshape(-1, 433,1)
train_x = torch.from_numpy(train_X)
train_y = torch.from_numpy(train_Y)
test_x = torch.from_numpy(test_X)
test_y=torch.from_numpy(test_Y)
val_x=torch.from_numpy(val_X)
val_y = torch.from_numpy(val_Y)

1.1753951


In [ ]:
print(standardization())

In [38]:
class FFN(nn.Module):
    def __init__(self):
        super(FFN, self).__init__()
        self.ffn=nn.Linear(20,1)
    def forward(self, x):  #
        out = self.ffn(x)
        return out

In [39]:
model=FFN().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for i in range(num_epochs):
    output=model(train_x.to(device))
    loss=criterion(output,train_y.to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    val_out=model(val_x.to(device))
    val_loss=criterion(val_out,val_y.to(device))
    print('Epoch: {}, Loss: {:.5f},val_Loss: {:.5f}'.format(i+1, loss,val_loss))

test_out=model(test_x.to(device))
test_loss=criterion(test_out,test_y.to(device))
print('test_Loss: {:.5f}'.format(test_loss))

Epoch: 1, Loss: 1.34900,val_Loss: 1.62079
Epoch: 2, Loss: 1.24452,val_Loss: 1.51119
Epoch: 3, Loss: 1.15535,val_Loss: 1.42058
Epoch: 4, Loss: 1.08158,val_Loss: 1.34879
Epoch: 5, Loss: 1.02309,val_Loss: 1.29522
Epoch: 6, Loss: 0.97939,val_Loss: 1.25876
Epoch: 7, Loss: 0.94959,val_Loss: 1.23764
Epoch: 8, Loss: 0.93226,val_Loss: 1.22943
Epoch: 9, Loss: 0.92546,val_Loss: 1.23111
Epoch: 10, Loss: 0.92674,val_Loss: 1.23924
Epoch: 11, Loss: 0.93333,val_Loss: 1.25040
Epoch: 12, Loss: 0.94246,val_Loss: 1.26154
Epoch: 13, Loss: 0.95165,val_Loss: 1.27032
Epoch: 14, Loss: 0.95902,val_Loss: 1.27534
Epoch: 15, Loss: 0.96341,val_Loss: 1.27605
Epoch: 16, Loss: 0.96437,val_Loss: 1.27267
Epoch: 17, Loss: 0.96205,val_Loss: 1.26593
Epoch: 18, Loss: 0.95704,val_Loss: 1.25686
Epoch: 19, Loss: 0.95016,val_Loss: 1.24662
Epoch: 20, Loss: 0.94233,val_Loss: 1.23629
Epoch: 21, Loss: 0.93443,val_Loss: 1.22681
Epoch: 22, Loss: 0.92721,val_Loss: 1.21888
Epoch: 23, Loss: 0.92122,val_Loss: 1.21292
Epoch: 24, Loss: 0.9